# Comparative analytics project between New York and Toronto - What makes a city unique.

#### This project aims to answer the question that what makes a city unique by providing a clearer picture about multiple aspects including their neighborhoods, their communities, venues, infrastructures and so on, and this project is for all travel-lovers, urbanologists, and anyone who lives in or is interested in these two cities. Hopefully I will find something interesting.


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

## Download and Explore New York Dataset

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [10]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [11]:
CLIENT_ID = 'LVR251M4E2M5F5SUBZZNISUPIIRXAC2AV1P1POTZUDSBVDP2' # your Foursquare ID
CLIENT_SECRET = '0VJC2SHNRFK5GEYLEGBQ43PEOI2YDLW4T4ZG1QGSI1P5T3AM' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LVR251M4E2M5F5SUBZZNISUPIIRXAC2AV1P1POTZUDSBVDP2
CLIENT_SECRET:0VJC2SHNRFK5GEYLEGBQ43PEOI2YDLW4T4ZG1QGSI1P5T3AM


In [30]:
address = 'Bushwick, Brooklyn, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.6942696 -73.9187482


In [31]:
search_query = 'University'
radius = 50000
print(search_query + ' .... OK!')

University .... OK!


In [32]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LVR251M4E2M5F5SUBZZNISUPIIRXAC2AV1P1POTZUDSBVDP2&client_secret=0VJC2SHNRFK5GEYLEGBQ43PEOI2YDLW4T4ZG1QGSI1P5T3AM&ll=40.6942696,-73.9187482&v=20180604&query=University&radius=50000&limit=30'

In [33]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e90c5ad006dce001beabfd5'},
 'response': {'venues': [{'id': '4227a500f964a520c51f1fe3',
    'name': 'Columbia University',
    'location': {'address': '116th St & Broadway',
     'lat': 40.80761943231824,
     'lng': -73.96236672844265,
     'distance': 13143,
     'postalCode': '10027',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['116th St & Broadway',
      'New York, NY 10027',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ae941735',
      'name': 'University',
      'pluralName': 'Universities',
      'shortName': 'University',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/default_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'id': '56742660'},
    'referralId': 'v-1586546383',
    'hasPerk': False},
   {'id': '5058de7be4b007870dd7ad7a',
    'name': 'New York University',
    'location': {'addr

In [34]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1ae941735', 'name': 'U...",False,4227a500f964a520c51f1fe3,116th St & Broadway,US,New York,United States,NaN,13143,"[116th St & Broadway, New York, NY 10027, Unit...",NaN,40.807619,-73.962367,NaN,10027,NY,Columbia University,v-1586546383,56742660
1,"[{'id': '4bf58dd8d48988d1ae941735', 'name': 'U...",False,5058de7be4b007870dd7ad7a,25 W 4th St,US,New York,United States,NaN,7401,"[25 W 4th St, New York, NY 10012, United States]","[{'label': 'display', 'lat': 40.72951795621751...",40.729518,-73.993122,NaN,10012,NY,New York University,v-1586546383,NaN
2,"[{'id': '4bf58dd8d48988d1ae941735', 'name': 'U...",False,4acccb59f964a52091c920e3,1230 York Ave,US,New York,United States,at E 66th St,8264,"[1230 York Ave (at E 66th St), New York, NY 10...","[{'label': 'display', 'lat': 40.76284692200573...",40.762847,-73.956277,NaN,10065,NY,The Rockefeller University,v-1586546383,NaN
3,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",False,4f7a34d3e4b01c9eb32ad618,NaN,US,Newark,United States,NaN,22875,"[Newark, NJ 07103, United States]",NaN,40.738434,-74.183529,NaN,07103,NJ,University Heights Newark,v-1586546383,NaN
4,"[{'id': '4bf58dd8d48988d198941735', 'name': 'C...",False,4ee7d0b25c5c04b331b036f5,65 5th Ave,US,New York,United States,14th St,7820,"[65 5th Ave (14th St), New York, NY 10003, Uni...","[{'label': 'display', 'lat': 40.73550608722417...",40.735506,-73.993784,NaN,10003,NY,New School University Center,v-1586546383,NaN


In [35]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Columbia University,University,116th St & Broadway,US,New York,United States,NaN,13143,"[116th St & Broadway, New York, NY 10027, Unit...",NaN,40.807619,-73.962367,NaN,10027,NY,4227a500f964a520c51f1fe3
1,New York University,University,25 W 4th St,US,New York,United States,NaN,7401,"[25 W 4th St, New York, NY 10012, United States]","[{'label': 'display', 'lat': 40.72951795621751...",40.729518,-73.993122,NaN,10012,NY,5058de7be4b007870dd7ad7a
2,The Rockefeller University,University,1230 York Ave,US,New York,United States,at E 66th St,8264,"[1230 York Ave (at E 66th St), New York, NY 10...","[{'label': 'display', 'lat': 40.76284692200573...",40.762847,-73.956277,NaN,10065,NY,4acccb59f964a52091c920e3
3,University Heights Newark,Neighborhood,NaN,US,Newark,United States,NaN,22875,"[Newark, NJ 07103, United States]",NaN,40.738434,-74.183529,NaN,07103,NJ,4f7a34d3e4b01c9eb32ad618
4,New School University Center,College Academic Building,65 5th Ave,US,New York,United States,14th St,7820,"[65 5th Ave (14th St), New York, NY 10003, Uni...","[{'label': 'display', 'lat': 40.73550608722417...",40.735506,-73.993784,NaN,10003,NY,4ee7d0b25c5c04b331b036f5
5,Pace University,University,1 Pace Plz,US,New York,United States,btwn Nassau & Gold St,7566,"[1 Pace Plz (btwn Nassau & Gold St), New York,...","[{'label': 'display', 'lat': 40.71072012218008...",40.710720,-74.005743,NaN,10038,NY,49e7c7c6f964a5201b651fe3
6,Brookdale University Hospital Medical Center,Hospital,1 Brookdale Plz,US,Brooklyn,United States,Linden Blvd,4418,"[1 Brookdale Plz (Linden Blvd), Brooklyn, NY 1...","[{'label': 'display', 'lat': 40.65487983303212...",40.654880,-73.912304,NaN,11212,NY,4b9a41dcf964a520c2a735e3
7,University Floral Design,Flower Shop,University,US,New York,United States,at 10th St.,7646,"[University (at 10th St.), New York, NY, Unite...","[{'label': 'display', 'lat': 40.73272692599926...",40.732727,-73.993841,NaN,NaN,NY,4cab867514c33704285beb3b
8,University Heights,Neighborhood,NaN,US,Bronx,United States,NaN,18659,"[Bronx, NY 10468, United States]",NaN,40.861744,-73.909336,NaN,10468,NY,4d51ee42bfc260fcd404c726
9,University Club,Social Club,1 W 54th St,US,New York,United States,at 5th Ave,8868,"[1 W 54th St (at 5th Ave), New York, NY 10019,...","[{'label': 'display', 'lat': 40.76121433933412...",40.761214,-73.975749,Midtown East,10019,NY,437d1980f964a520a82a1fe3


In [36]:
df1 = dataframe_filtered[dataframe_filtered['categories']=='University']
df1.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Columbia University,University,116th St & Broadway,US,New York,United States,NaN,13143,"[116th St & Broadway, New York, NY 10027, Unit...",NaN,40.807619,-73.962367,NaN,10027,NY,4227a500f964a520c51f1fe3
1,New York University,University,25 W 4th St,US,New York,United States,NaN,7401,"[25 W 4th St, New York, NY 10012, United States]","[{'label': 'display', 'lat': 40.72951795621751...",40.729518,-73.993122,NaN,10012,NY,5058de7be4b007870dd7ad7a
2,The Rockefeller University,University,1230 York Ave,US,New York,United States,at E 66th St,8264,"[1230 York Ave (at E 66th St), New York, NY 10...","[{'label': 'display', 'lat': 40.76284692200573...",40.762847,-73.956277,NaN,10065,NY,4acccb59f964a52091c920e3
5,Pace University,University,1 Pace Plz,US,New York,United States,btwn Nassau & Gold St,7566,"[1 Pace Plz (btwn Nassau & Gold St), New York,...","[{'label': 'display', 'lat': 40.71072012218008...",40.710720,-74.005743,NaN,10038,NY,49e7c7c6f964a5201b651fe3
10,Long Island University,University,1 University Plz,US,Brooklyn,United States,at Flatbush & Dekalb Ave.,5302,"[1 University Plz (at Flatbush & Dekalb Ave.),...","[{'label': 'display', 'lat': 40.69076482237715...",40.690765,-73.981399,NaN,11201,NY,4aef89aef964a52027d921e3


In [37]:
df1.name

0                       Columbia University
1                       New York University
2                The Rockefeller University
5                           Pace University
10                   Long Island University
11      Fordham University - Lincoln Center
12                    St. John's University
26    Teachers College, Columbia University
27                 Saint Peter's University
Name: name, dtype: object

In [38]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel


# add the UNIVERSITY as blue circle markers
for lat, lng, label in zip(df1.lat, df1.lng, df1.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [52]:
search_query = 'Hospital'
radius = 25000
print(search_query + ' .... OK!')


Hospital .... OK!


In [53]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LVR251M4E2M5F5SUBZZNISUPIIRXAC2AV1P1POTZUDSBVDP2&client_secret=0VJC2SHNRFK5GEYLEGBQ43PEOI2YDLW4T4ZG1QGSI1P5T3AM&ll=40.6942696,-73.9187482&v=20180604&query=Hospital&radius=25000&limit=30'

In [54]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e90c7681d67cb001b7fad30'},
 'response': {'venues': [{'id': '4a5bece4f964a52000bc1fe3',
    'name': 'New York Presbyterian Hospital Weill Cornell Medical Center',
    'location': {'address': '525 E 68th St Fl 6',
     'crossStreet': 'at York Ave',
     'lat': 40.764802043047226,
     'lng': -73.95405493770409,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.764802043047226,
       'lng': -73.95405493770409}],
     'distance': 8397,
     'postalCode': '10065',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['525 E 68th St Fl 6 (at York Ave)',
      'New York, NY 10065',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d196941735',
      'name': 'Hospital',
      'pluralName': 'Hospitals',
      'shortName': 'Hospital',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
       'suffix': '.png'},
      'primary': Tru

In [55]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe_2 = json_normalize(venues)
dataframe_2.head(20)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4a5bece4f964a52000bc1fe3,525 E 68th St Fl 6,US,New York,United States,at York Ave,8397,"[525 E 68th St Fl 6 (at York Ave), New York, N...","[{'label': 'display', 'lat': 40.76480204304722...",40.764802,-73.954055,NaN,10065,NY,New York Presbyterian Hospital Weill Cornell M...,v-1586546792,NaN
1,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4a43baaaf964a520b9a61fe3,462 1st Ave,US,New York,United States,between E 26th & E 29th St.,7034,"[462 1st Ave (between E 26th & E 29th St.), Ne...","[{'label': 'display', 'lat': 40.73972514676565...",40.739725,-73.976661,NaN,10016,NY,Bellevue Hospital Center,v-1586546792,NaN
2,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4a9d8f44f964a5207e3820e3,1468 Madison Avenue,US,New York,United States,at E 101st St,11034,"[1468 Madison Avenue (at E 101st St), New York...","[{'label': 'display', 'lat': 40.7899953, 'lng'...",40.789995,-73.952725,NaN,10029,NY,The Mount Sinai Hospital,v-1586546792,NaN
3,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4a78348ff964a52046e51fe3,535 E 70th St,US,New York,United States,at York Ave,8414,"[535 E 70th St (at York Ave), New York, NY 100...",NaN,40.765348,-73.952689,NaN,10021,NY,Hospital for Special Surgery,v-1586546792,NaN
4,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4a386563f964a520f29e1fe3,506 6th St,US,Brooklyn,United States,7th Ave.,5841,"[506 6th St (7th Ave.), Brooklyn, NY 11215, Un...","[{'label': 'display', 'lat': 40.6677145, 'lng'...",40.667715,-73.978424,"Park Slope, Brooklyn, NY",11215,NY,NewYork-Presbyterian Brooklyn Methodist Hospital,v-1586546792,NaN
5,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4b688c4df964a520357f2be3,451 Clarkson Ave,US,Brooklyn,United States,NaN,4661,"[451 Clarkson Ave, Brooklyn, NY 11203, United ...","[{'label': 'display', 'lat': 40.65671312078148...",40.656713,-73.943181,NaN,11203,NY,Kings County Hospital Center,v-1586546792,NaN
6,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4acbc102f964a5204bc520e3,470 Clarkson Ave,US,Brooklyn,United States,E 37th St,4846,"[470 Clarkson Ave (E 37th St), Brooklyn, NY 11...","[{'label': 'display', 'lat': 40.65528203674459...",40.655282,-73.944303,NaN,11203,NY,University Hospital of Brooklyn,v-1586546792,NaN
7,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4aed5b59f964a52087cd21e3,121 Dekalb Ave,US,Brooklyn,United States,at Ashland Pl.,5001,"[121 Dekalb Ave (at Ashland Pl.), Brooklyn, NY...","[{'label': 'display', 'lat': 40.69023812388737...",40.690238,-73.977771,NaN,11201,NY,The Brooklyn Hospital Center,v-1586546792,554423167
8,"[{'id': '4bf58dd8d48988d194941735', 'name': 'E...",False,4ca9e2f8d971b1f774c822e1,NaN,US,Brooklyn,United States,NaN,1073,"[Brooklyn, NY 11237, United States]","[{'label': 'display', 'lat': 40.70385920234893...",40.703859,-73.917450,NaN,11237,NY,Wyckoff Hospital ER,v-1586546792,NaN
9,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4a82ef0af964a52092f91fe3,170 William St,US,New York,United States,at Gold St,7464,"[170 William St (at Gold St), New York, NY 100...","[{'label': 'display', 'lat': 40.7099166, 'lng'...",40.709917,-74.004758,NaN,10038,NY,NewYork-Presbyterian-Lower Manhattan Hospital,v-1586546792,NaN


In [56]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe_2.columns if col.startswith('location.')] + ['id']
dataframe_filtered_2 = dataframe_2.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_2['categories'] = dataframe_filtered_2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_2.columns = [column.split('.')[-1] for column in dataframe_filtered_2.columns]

dataframe_filtered_2

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,New York Presbyterian Hospital Weill Cornell M...,Hospital,525 E 68th St Fl 6,US,New York,United States,at York Ave,8397,"[525 E 68th St Fl 6 (at York Ave), New York, N...","[{'label': 'display', 'lat': 40.76480204304722...",40.764802,-73.954055,NaN,10065,NY,4a5bece4f964a52000bc1fe3
1,Bellevue Hospital Center,Hospital,462 1st Ave,US,New York,United States,between E 26th & E 29th St.,7034,"[462 1st Ave (between E 26th & E 29th St.), Ne...","[{'label': 'display', 'lat': 40.73972514676565...",40.739725,-73.976661,NaN,10016,NY,4a43baaaf964a520b9a61fe3
2,The Mount Sinai Hospital,Hospital,1468 Madison Avenue,US,New York,United States,at E 101st St,11034,"[1468 Madison Avenue (at E 101st St), New York...","[{'label': 'display', 'lat': 40.7899953, 'lng'...",40.789995,-73.952725,NaN,10029,NY,4a9d8f44f964a5207e3820e3
3,Hospital for Special Surgery,Hospital,535 E 70th St,US,New York,United States,at York Ave,8414,"[535 E 70th St (at York Ave), New York, NY 100...",NaN,40.765348,-73.952689,NaN,10021,NY,4a78348ff964a52046e51fe3
4,NewYork-Presbyterian Brooklyn Methodist Hospital,Hospital,506 6th St,US,Brooklyn,United States,7th Ave.,5841,"[506 6th St (7th Ave.), Brooklyn, NY 11215, Un...","[{'label': 'display', 'lat': 40.6677145, 'lng'...",40.667715,-73.978424,"Park Slope, Brooklyn, NY",11215,NY,4a386563f964a520f29e1fe3
5,Kings County Hospital Center,Hospital,451 Clarkson Ave,US,Brooklyn,United States,NaN,4661,"[451 Clarkson Ave, Brooklyn, NY 11203, United ...","[{'label': 'display', 'lat': 40.65671312078148...",40.656713,-73.943181,NaN,11203,NY,4b688c4df964a520357f2be3
6,University Hospital of Brooklyn,Hospital,470 Clarkson Ave,US,Brooklyn,United States,E 37th St,4846,"[470 Clarkson Ave (E 37th St), Brooklyn, NY 11...","[{'label': 'display', 'lat': 40.65528203674459...",40.655282,-73.944303,NaN,11203,NY,4acbc102f964a5204bc520e3
7,The Brooklyn Hospital Center,Hospital,121 Dekalb Ave,US,Brooklyn,United States,at Ashland Pl.,5001,"[121 Dekalb Ave (at Ashland Pl.), Brooklyn, NY...","[{'label': 'display', 'lat': 40.69023812388737...",40.690238,-73.977771,NaN,11201,NY,4aed5b59f964a52087cd21e3
8,Wyckoff Hospital ER,Emergency Room,NaN,US,Brooklyn,United States,NaN,1073,"[Brooklyn, NY 11237, United States]","[{'label': 'display', 'lat': 40.70385920234893...",40.703859,-73.917450,NaN,11237,NY,4ca9e2f8d971b1f774c822e1
9,NewYork-Presbyterian-Lower Manhattan Hospital,Hospital,170 William St,US,New York,United States,at Gold St,7464,"[170 William St (at Gold St), New York, NY 100...","[{'label': 'display', 'lat': 40.7099166, 'lng'...",40.709917,-74.004758,NaN,10038,NY,4a82ef0af964a52092f91fe3


In [58]:
df2 = dataframe_filtered_2[dataframe_filtered_2['categories']=='Hospital']
df2.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,New York Presbyterian Hospital Weill Cornell M...,Hospital,525 E 68th St Fl 6,US,New York,United States,at York Ave,8397,"[525 E 68th St Fl 6 (at York Ave), New York, N...","[{'label': 'display', 'lat': 40.76480204304722...",40.764802,-73.954055,NaN,10065,NY,4a5bece4f964a52000bc1fe3
1,Bellevue Hospital Center,Hospital,462 1st Ave,US,New York,United States,between E 26th & E 29th St.,7034,"[462 1st Ave (between E 26th & E 29th St.), Ne...","[{'label': 'display', 'lat': 40.73972514676565...",40.739725,-73.976661,NaN,10016,NY,4a43baaaf964a520b9a61fe3
2,The Mount Sinai Hospital,Hospital,1468 Madison Avenue,US,New York,United States,at E 101st St,11034,"[1468 Madison Avenue (at E 101st St), New York...","[{'label': 'display', 'lat': 40.7899953, 'lng'...",40.789995,-73.952725,NaN,10029,NY,4a9d8f44f964a5207e3820e3
3,Hospital for Special Surgery,Hospital,535 E 70th St,US,New York,United States,at York Ave,8414,"[535 E 70th St (at York Ave), New York, NY 100...",NaN,40.765348,-73.952689,NaN,10021,NY,4a78348ff964a52046e51fe3
4,NewYork-Presbyterian Brooklyn Methodist Hospital,Hospital,506 6th St,US,Brooklyn,United States,7th Ave.,5841,"[506 6th St (7th Ave.), Brooklyn, NY 11215, Un...","[{'label': 'display', 'lat': 40.6677145, 'lng'...",40.667715,-73.978424,"Park Slope, Brooklyn, NY",11215,NY,4a386563f964a520f29e1fe3
5,Kings County Hospital Center,Hospital,451 Clarkson Ave,US,Brooklyn,United States,NaN,4661,"[451 Clarkson Ave, Brooklyn, NY 11203, United ...","[{'label': 'display', 'lat': 40.65671312078148...",40.656713,-73.943181,NaN,11203,NY,4b688c4df964a520357f2be3
6,University Hospital of Brooklyn,Hospital,470 Clarkson Ave,US,Brooklyn,United States,E 37th St,4846,"[470 Clarkson Ave (E 37th St), Brooklyn, NY 11...","[{'label': 'display', 'lat': 40.65528203674459...",40.655282,-73.944303,NaN,11203,NY,4acbc102f964a5204bc520e3
7,The Brooklyn Hospital Center,Hospital,121 Dekalb Ave,US,Brooklyn,United States,at Ashland Pl.,5001,"[121 Dekalb Ave (at Ashland Pl.), Brooklyn, NY...","[{'label': 'display', 'lat': 40.69023812388737...",40.690238,-73.977771,NaN,11201,NY,4aed5b59f964a52087cd21e3
9,NewYork-Presbyterian-Lower Manhattan Hospital,Hospital,170 William St,US,New York,United States,at Gold St,7464,"[170 William St (at Gold St), New York, NY 100...","[{'label': 'display', 'lat': 40.7099166, 'lng'...",40.709917,-74.004758,NaN,10038,NY,4a82ef0af964a52092f91fe3
10,Brookdale University Hospital Medical Center,Hospital,1 Brookdale Plz,US,Brooklyn,United States,Linden Blvd,4418,"[1 Brookdale Plz (Linden Blvd), Brooklyn, NY 1...","[{'label': 'display', 'lat': 40.65487983303212...",40.654880,-73.912304,NaN,11212,NY,4b9a41dcf964a520c2a735e3


In [59]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel


# add the UNIVERSITY as blue circle markers
for lat, lng, label in zip(df2.lat, df2.lng, df2.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [60]:
search_query = 'Subway Station'
radius = 25000
print(search_query + ' .... OK!')



Subway Station .... OK!


In [61]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LVR251M4E2M5F5SUBZZNISUPIIRXAC2AV1P1POTZUDSBVDP2&client_secret=0VJC2SHNRFK5GEYLEGBQ43PEOI2YDLW4T4ZG1QGSI1P5T3AM&ll=40.6942696,-73.9187482&v=20180604&query=Subway Station&radius=25000&limit=30'

In [62]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e90c8a03907e7001b7ee234'},
 'response': {'venues': [{'id': '4ec098836c25dfd982054037',
    'name': '72nd St Subway Station Newsstand',
    'location': {'address': 'Broadway btwn 72nd & 73rd',
     'crossStreet': 'Amsterdam Ave.',
     'lat': 40.77884622907713,
     'lng': -73.98193362840085,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.77884622907713,
       'lng': -73.98193362840085}],
     'distance': 10818,
     'postalCode': '10023',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['Broadway btwn 72nd & 73rd (Amsterdam Ave.)',
      'New York, NY 10023',
      'United States']},
    'categories': [{'id': '4f04ad622fb6e1c99f3db0b9',
      'name': 'Newsstand',
      'pluralName': 'Newsstands',
      'shortName': 'Newsstand',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/newsstand_',
       'suffix': '.png'},
      'primary': True}],
 

In [63]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe_3 = json_normalize(venues)
dataframe_3.head(20)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4f04ad622fb6e1c99f3db0b9', 'name': 'N...",False,4ec098836c25dfd982054037,Broadway btwn 72nd & 73rd,US,New York,United States,Amsterdam Ave.,10818,"[Broadway btwn 72nd & 73rd (Amsterdam Ave.), N...","[{'label': 'display', 'lat': 40.77884622907713...",40.778846,-73.981934,NaN,10023,NY,72nd St Subway Station Newsstand,v-1586546958,65043329
1,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,4a4d10fbf964a52082ad1fe3,Van Sinderen Ave,US,Brooklyn,United States,btwn Eastern Pkwy & Fulton St,2156,[Van Sinderen Ave (btwn Eastern Pkwy & Fulton ...,"[{'label': 'display', 'lat': 40.67842894820232...",40.678429,-73.904036,NaN,11233,NY,MTA Subway - Broadway Junction (A/C/J/L/Z),v-1586546958,NaN
2,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",False,4e0cffc522711665f606edb4,Park Pl,US,Prospect Heights,United States,Carlton Ave,7914,"[Park Pl (Carlton Ave), Prospect Heights, NY 1...","[{'label': 'display', 'lat': 40.71392620399956...",40.713926,-74.008882,NaN,11217,NY,NYC Green Cart (7th Ave Subway Station),v-1586546958,NaN
3,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,4b0737c3f964a520e4f922e3,W 42nd St,US,New York,United States,btwn Broadway & 8th Ave,9004,"[W 42nd St (btwn Broadway & 8th Ave), New York...",NaN,40.755891,-73.987890,Theater District,10036,NY,MTA Subway - 42nd St/Times Square/Port Authori...,v-1586546958,NaN
4,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,4b52cd44f964a5204d8827e3,Myrtle Ave,US,Brooklyn,United States,at Wyckoff Ave,863,"[Myrtle Ave (at Wyckoff Ave), Brooklyn, NY 112...","[{'label': 'display', 'lat': 40.69974909380805...",40.699749,-73.911511,NaN,11237,NY,MTA Subway - Myrtle/Wyckoff Ave (L/M),v-1586546958,NaN
5,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,4b5d9f0df964a520d06329e3,Kosciuszko St,US,Brooklyn,United States,Broadway,819,"[Kosciuszko St (Broadway), Brooklyn, NY 11221,...","[{'label': 'display', 'lat': 40.69306796133083...",40.693068,-73.928325,NaN,11221,NY,MTA Subway - Kosciuszko St (J),v-1586546958,NaN
6,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,526841c9498e6f7c90d7d3f8,NaN,US,North Bergen,United States,NaN,13597,"[North Bergen, NJ, United States]","[{'label': 'display', 'lat': 40.769938, 'lng':...",40.769938,-74.045290,NaN,NaN,NJ,Rector Street Subway Station,v-1586546958,NaN
7,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,4acfc371f964a520d2d520e3,Flatbush Ave,US,Brooklyn,United States,btwn Atlantic Ave & Pacific St,5165,[Flatbush Ave (btwn Atlantic Ave & Pacific St)...,"[{'label': 'display', 'lat': 40.6841038660902,...",40.684104,-73.978464,NaN,11217,NY,MTA Subway - Atlantic Ave/Barclays Center (B/D...,v-1586546958,NaN
8,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,4b3a1855f964a520026125e3,Jay St,US,Brooklyn,United States,at Willoughby St,5787,"[Jay St (at Willoughby St), Brooklyn, NY 11201...","[{'label': 'display', 'lat': 40.69213295884398...",40.692133,-73.987263,NaN,11201,NY,MTA Subway - Jay St/MetroTech (A/C/F/R),v-1586546958,NaN
9,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,4bb7b1153db7b713bd65209a,1429 Myrtle Ave,US,Brooklyn,United States,at Knickerbocker Ave,500,"[1429 Myrtle Ave (at Knickerbocker Ave), Brook...","[{'label': 'display', 'lat': 40.698750642989, ...",40.698751,-73.919285,NaN,11237,NY,MTA Subway - Knickerbocker Ave (M),v-1586546958,NaN


In [64]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe_3.columns if col.startswith('location.')] + ['id']
dataframe_filtered_3 = dataframe_3.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_3['categories'] = dataframe_filtered_3.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_3.columns = [column.split('.')[-1] for column in dataframe_filtered_3.columns]

dataframe_filtered_3

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,72nd St Subway Station Newsstand,Newsstand,Broadway btwn 72nd & 73rd,US,New York,United States,Amsterdam Ave.,10818,"[Broadway btwn 72nd & 73rd (Amsterdam Ave.), N...","[{'label': 'display', 'lat': 40.77884622907713...",40.778846,-73.981934,NaN,10023,NY,4ec098836c25dfd982054037
1,MTA Subway - Broadway Junction (A/C/J/L/Z),Metro Station,Van Sinderen Ave,US,Brooklyn,United States,btwn Eastern Pkwy & Fulton St,2156,[Van Sinderen Ave (btwn Eastern Pkwy & Fulton ...,"[{'label': 'display', 'lat': 40.67842894820232...",40.678429,-73.904036,NaN,11233,NY,4a4d10fbf964a52082ad1fe3
2,NYC Green Cart (7th Ave Subway Station),Food Truck,Park Pl,US,Prospect Heights,United States,Carlton Ave,7914,"[Park Pl (Carlton Ave), Prospect Heights, NY 1...","[{'label': 'display', 'lat': 40.71392620399956...",40.713926,-74.008882,NaN,11217,NY,4e0cffc522711665f606edb4
3,MTA Subway - 42nd St/Times Square/Port Authori...,Metro Station,W 42nd St,US,New York,United States,btwn Broadway & 8th Ave,9004,"[W 42nd St (btwn Broadway & 8th Ave), New York...",NaN,40.755891,-73.987890,Theater District,10036,NY,4b0737c3f964a520e4f922e3
4,MTA Subway - Myrtle/Wyckoff Ave (L/M),Metro Station,Myrtle Ave,US,Brooklyn,United States,at Wyckoff Ave,863,"[Myrtle Ave (at Wyckoff Ave), Brooklyn, NY 112...","[{'label': 'display', 'lat': 40.69974909380805...",40.699749,-73.911511,NaN,11237,NY,4b52cd44f964a5204d8827e3
5,MTA Subway - Kosciuszko St (J),Metro Station,Kosciuszko St,US,Brooklyn,United States,Broadway,819,"[Kosciuszko St (Broadway), Brooklyn, NY 11221,...","[{'label': 'display', 'lat': 40.69306796133083...",40.693068,-73.928325,NaN,11221,NY,4b5d9f0df964a520d06329e3
6,Rector Street Subway Station,Metro Station,NaN,US,North Bergen,United States,NaN,13597,"[North Bergen, NJ, United States]","[{'label': 'display', 'lat': 40.769938, 'lng':...",40.769938,-74.045290,NaN,NaN,NJ,526841c9498e6f7c90d7d3f8
7,MTA Subway - Atlantic Ave/Barclays Center (B/D...,Metro Station,Flatbush Ave,US,Brooklyn,United States,btwn Atlantic Ave & Pacific St,5165,[Flatbush Ave (btwn Atlantic Ave & Pacific St)...,"[{'label': 'display', 'lat': 40.6841038660902,...",40.684104,-73.978464,NaN,11217,NY,4acfc371f964a520d2d520e3
8,MTA Subway - Jay St/MetroTech (A/C/F/R),Metro Station,Jay St,US,Brooklyn,United States,at Willoughby St,5787,"[Jay St (at Willoughby St), Brooklyn, NY 11201...","[{'label': 'display', 'lat': 40.69213295884398...",40.692133,-73.987263,NaN,11201,NY,4b3a1855f964a520026125e3
9,MTA Subway - Knickerbocker Ave (M),Metro Station,1429 Myrtle Ave,US,Brooklyn,United States,at Knickerbocker Ave,500,"[1429 Myrtle Ave (at Knickerbocker Ave), Brook...","[{'label': 'display', 'lat': 40.698750642989, ...",40.698751,-73.919285,NaN,11237,NY,4bb7b1153db7b713bd65209a


In [66]:
df3 = dataframe_filtered_3[dataframe_filtered_3['categories']=='Metro Station']
df3.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
1,MTA Subway - Broadway Junction (A/C/J/L/Z),Metro Station,Van Sinderen Ave,US,Brooklyn,United States,btwn Eastern Pkwy & Fulton St,2156,[Van Sinderen Ave (btwn Eastern Pkwy & Fulton ...,"[{'label': 'display', 'lat': 40.67842894820232...",40.678429,-73.904036,NaN,11233,NY,4a4d10fbf964a52082ad1fe3
3,MTA Subway - 42nd St/Times Square/Port Authori...,Metro Station,W 42nd St,US,New York,United States,btwn Broadway & 8th Ave,9004,"[W 42nd St (btwn Broadway & 8th Ave), New York...",NaN,40.755891,-73.987890,Theater District,10036,NY,4b0737c3f964a520e4f922e3
4,MTA Subway - Myrtle/Wyckoff Ave (L/M),Metro Station,Myrtle Ave,US,Brooklyn,United States,at Wyckoff Ave,863,"[Myrtle Ave (at Wyckoff Ave), Brooklyn, NY 112...","[{'label': 'display', 'lat': 40.69974909380805...",40.699749,-73.911511,NaN,11237,NY,4b52cd44f964a5204d8827e3
5,MTA Subway - Kosciuszko St (J),Metro Station,Kosciuszko St,US,Brooklyn,United States,Broadway,819,"[Kosciuszko St (Broadway), Brooklyn, NY 11221,...","[{'label': 'display', 'lat': 40.69306796133083...",40.693068,-73.928325,NaN,11221,NY,4b5d9f0df964a520d06329e3
6,Rector Street Subway Station,Metro Station,NaN,US,North Bergen,United States,NaN,13597,"[North Bergen, NJ, United States]","[{'label': 'display', 'lat': 40.769938, 'lng':...",40.769938,-74.045290,NaN,NaN,NJ,526841c9498e6f7c90d7d3f8
7,MTA Subway - Atlantic Ave/Barclays Center (B/D...,Metro Station,Flatbush Ave,US,Brooklyn,United States,btwn Atlantic Ave & Pacific St,5165,[Flatbush Ave (btwn Atlantic Ave & Pacific St)...,"[{'label': 'display', 'lat': 40.6841038660902,...",40.684104,-73.978464,NaN,11217,NY,4acfc371f964a520d2d520e3
8,MTA Subway - Jay St/MetroTech (A/C/F/R),Metro Station,Jay St,US,Brooklyn,United States,at Willoughby St,5787,"[Jay St (at Willoughby St), Brooklyn, NY 11201...","[{'label': 'display', 'lat': 40.69213295884398...",40.692133,-73.987263,NaN,11201,NY,4b3a1855f964a520026125e3
9,MTA Subway - Knickerbocker Ave (M),Metro Station,1429 Myrtle Ave,US,Brooklyn,United States,at Knickerbocker Ave,500,"[1429 Myrtle Ave (at Knickerbocker Ave), Brook...","[{'label': 'display', 'lat': 40.698750642989, ...",40.698751,-73.919285,NaN,11237,NY,4bb7b1153db7b713bd65209a
10,MTA Subway - 14th St/Union Square (4/5/6/L/N/Q...,Metro Station,7 E 14th St,US,New York,United States,at Broadway & Union Square,7537,"[7 E 14th St (at Broadway & Union Square), New...",NaN,40.734871,-73.990243,NaN,10003,NY,4ad8add6f964a520a61321e3
11,MTA Subway - Gates Ave (J/Z),Metro Station,Gates Ave,US,Brooklyn,United States,at Broadway,602,"[Gates Ave (at Broadway), Brooklyn, NY 11221, ...","[{'label': 'display', 'lat': 40.68975927343005...",40.689759,-73.922683,NaN,11221,NY,4b5284f5f964a520ce8027e3


In [67]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel


# add the UNIVERSITY as blue circle markers
for lat, lng, label in zip(df3.lat, df3.lng, df3.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Create a function to easily visualize the venue in toronto 

In [71]:
address = '33 Wanless Crescent, Toronto, ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)


search_query = 'University'
radius = 25000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
data_trt = json_normalize(venues)
    
filtered_columns = ['name', 'categories'] + [col for col in data_trt.columns if col.startswith('location.')] + ['id']
data_trt_final = data_trt.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
data_trt_final['categories'] = data_trt_final.apply(get_category_type, axis=1)

# clean column names by keeping only last term
data_trt_final.columns = [column.split('.')[-1] for column in data_trt_final.columns]
data_trt_final

    

43.7265911 -79.3908283


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,TTC Subway Line 1 - Yonge-University-Spadina Line,Metro Station,Yonge - University,CA,Toronto,Canada,TTC Subway Line#2,1509,"[Yonge - University (TTC Subway Line#2), Toron...","[{'label': 'display', 'lat': 43.71485231875422...",43.714852,-79.400210,NaN,ON,4d1ea7cd21a72c0f8e132ee3
1,University of Toronto,University,27 King's College Cir,CA,Toronto,Canada,at King's College Rd,7144,"[27 King's College Cir (at King's College Rd),...","[{'label': 'display', 'lat': 43.6624934706167,...",43.662493,-79.395220,M5S 1A1,ON,4ad4c05ef964a52097f620e3
2,"Glendon Campus, York University",University,2275 Bayview Ave,CA,Toronto,Canada,Bayview Ave & Lawrence Ave,282,[2275 Bayview Ave (Bayview Ave & Lawrence Ave)...,"[{'label': 'display', 'lat': 43.72698108205825...",43.726981,-79.394296,NaN,ON,5159aa97e4b0b91a42c29828
3,University College,University,15 King's College Circle,CA,Toronto,Canada,University of Toronto,7125,[15 King's College Circle (University of Toron...,"[{'label': 'display', 'lat': 43.66269528389713...",43.662695,-79.396061,M5S 3H7,ON,4ad893f7f964a5207c1221e3
4,University at St. Michael's College,University,St Joseph St,CA,Toronto,Canada,Bay,6777,"[St Joseph St (Bay), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.66571145228362...",43.665711,-79.389909,NaN,ON,4b957655f964a520d2a334e3
5,University of Toronto Arts Centre,Art Gallery,15 Kings College Circle,CA,Toronto,Canada,NaN,7020,"[15 Kings College Circle, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.66360538340076...",43.663605,-79.395159,NaN,ON,4bfd97014cf820a1c0b0ecf4
6,University College Residence Office,University,79 St. George street,CA,Toronto,Canada,NaN,7065,"[79 St. George street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.66333075902387...",43.663331,-79.397947,NaN,ON,4bbe3fdd1416a5931ce3f33c
7,University Avenue Plaza,Plaza,University Avenue,CA,Toronto,Canada,btwn Richmond & College,8231,"[University Avenue (btwn Richmond & College), ...","[{'label': 'display', 'lat': 43.65269470686765...",43.652695,-79.387218,NaN,ON,4e6565fb1850e238d7048495
8,University Eye Clinic,Doctor's Office,"700 University Ave, Suite 3",CA,Toronto,Canada,College Street,7493,"[700 University Ave, Suite 3 (College Street),...","[{'label': 'display', 'lat': 43.65927999991696...",43.659280,-79.390281,M5G 1Z5,ON,4c48b7c1b5eec9b63200caa3
9,York University - Keele Campus,University,4700 Keele St.,CA,Toronto,Canada,NaN,10289,"[4700 Keele St., Toronto ON M3J 1P3, Canada]","[{'label': 'display', 'lat': 43.77210701043441...",43.772107,-79.502190,M3J 1P3,ON,4ad78121f964a5208b0b21e3


In [75]:
trt = data_trt_final[data_trt_final['categories']=='University']
trt.head(20)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
1,University of Toronto,University,27 King's College Cir,CA,Toronto,Canada,at King's College Rd,7144,"[27 King's College Cir (at King's College Rd),...","[{'label': 'display', 'lat': 43.6624934706167,...",43.662493,-79.395220,M5S 1A1,ON,4ad4c05ef964a52097f620e3
2,"Glendon Campus, York University",University,2275 Bayview Ave,CA,Toronto,Canada,Bayview Ave & Lawrence Ave,282,[2275 Bayview Ave (Bayview Ave & Lawrence Ave)...,"[{'label': 'display', 'lat': 43.72698108205825...",43.726981,-79.394296,NaN,ON,5159aa97e4b0b91a42c29828
3,University College,University,15 King's College Circle,CA,Toronto,Canada,University of Toronto,7125,[15 King's College Circle (University of Toron...,"[{'label': 'display', 'lat': 43.66269528389713...",43.662695,-79.396061,M5S 3H7,ON,4ad893f7f964a5207c1221e3
4,University at St. Michael's College,University,St Joseph St,CA,Toronto,Canada,Bay,6777,"[St Joseph St (Bay), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.66571145228362...",43.665711,-79.389909,NaN,ON,4b957655f964a520d2a334e3
6,University College Residence Office,University,79 St. George street,CA,Toronto,Canada,NaN,7065,"[79 St. George street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.66333075902387...",43.663331,-79.397947,NaN,ON,4bbe3fdd1416a5931ce3f33c
9,York University - Keele Campus,University,4700 Keele St.,CA,Toronto,Canada,NaN,10289,"[4700 Keele St., Toronto ON M3J 1P3, Canada]","[{'label': 'display', 'lat': 43.77210701043441...",43.772107,-79.502190,M3J 1P3,ON,4ad78121f964a5208b0b21e3
19,University of Guelph-Humber,University,207 Humber College Blvd.,CA,Toronto,Canada,at Hwy 27,17336,"[207 Humber College Blvd. (at Hwy 27), Toronto...","[{'label': 'display', 'lat': 43.72811409611923...",43.728114,-79.606333,M9W 5L7,ON,4add284df964a5205d6421e3
22,Ontario College of Art and Design University (...,University,100 McCaul St,CA,Toronto,Canada,at Dundas St W,8214,"[100 McCaul St (at Dundas St W), Toronto ON M5...","[{'label': 'display', 'lat': 43.65280251171013...",43.652803,-79.391074,M5T 1W1,ON,4ad4c064f964a52065f820e3


In [74]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='33 Wanless Crescent',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the UNIVERSITY as blue circle markers
for lat, lng, label in zip(trt.lat, trt.lng, trt.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map